In [44]:
# install pycaret
import pandas as pd
import nltk
from nltk.corpus import stopwords
from pycaret.anomaly import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors


In [13]:
df = pd.read_csv('all_movies.csv', delimiter=';')
df

,title_pt,title_en,year,rating,genre,sinopse
0,O Poderoso Chefão II,4. The Godfather Part II,1974,9.0,Policial,"Em 1950, Michael Corleone, agora à frente da f..."
1,12 Homens e uma Sentença,5. 12 Angry Men,1957,9.0,Policial,O julgamento de um assassinato em Nova Iorque ...
2,A Lista de Schindler,6. Schindler's List,1993,9.0,Biografia,Na Polônia ocupada pelos alemães durante a Seg...
3,O Senhor dos Anéis: O Retorno do Rei,7. The Lord of the Rings: The Return of the King,2003,9.0,Ação,Gandalf e Aragorn lideram o Mundo dos Homens c...
4,O Senhor dos Anéis: A Sociedade do Anel,9. The Lord of the Rings: The Fellowship of th...,2001,8.8,Ação,Um manso hobbit do Condado e oito companheiros...
...,...,...,...,...,...,...
431,Homem-aranha: Lotus,Spider-Man: Lotus,2023,3.4,Ação,"Após a morte de sua namorada, Peter Parker se ..."
432,The Marsh King's Daughter,The Marsh King's Daughter,2023,NaN,Policial,A woman seeks revenge against the man who kidn...
433,O Rei Macaco,The Monkey King,2023,5.8,Animação,Segue um macaco cheio de carisma e seu bastão ...
434,Gladiador,Gladiator,2000,8.5,Ação,Um general romano planeja a melhor vingança co...


In [14]:
df.isnull().sum()

title_pt     0
title_en     0
year         0
rating      14
genre        0
sinopse      0
dtype: int64

In [15]:
df.dropna(subset=['rating'], inplace=True)
df

,title_pt,title_en,year,rating,genre,sinopse
0,O Poderoso Chefão II,4. The Godfather Part II,1974,9.0,Policial,"Em 1950, Michael Corleone, agora à frente da f..."
1,12 Homens e uma Sentença,5. 12 Angry Men,1957,9.0,Policial,O julgamento de um assassinato em Nova Iorque ...
2,A Lista de Schindler,6. Schindler's List,1993,9.0,Biografia,Na Polônia ocupada pelos alemães durante a Seg...
3,O Senhor dos Anéis: O Retorno do Rei,7. The Lord of the Rings: The Return of the King,2003,9.0,Ação,Gandalf e Aragorn lideram o Mundo dos Homens c...
4,O Senhor dos Anéis: A Sociedade do Anel,9. The Lord of the Rings: The Fellowship of th...,2001,8.8,Ação,Um manso hobbit do Condado e oito companheiros...
...,...,...,...,...,...,...
430,Tudo em Todo o Lugar ao Mesmo Tempo,Everything Everywhere All at Once,2022,7.8,Ação,Uma imigrante chinesa de meia idade se envolve...
431,Homem-aranha: Lotus,Spider-Man: Lotus,2023,3.4,Ação,"Após a morte de sua namorada, Peter Parker se ..."
433,O Rei Macaco,The Monkey King,2023,5.8,Animação,Segue um macaco cheio de carisma e seu bastão ...
434,Gladiador,Gladiator,2000,8.5,Ação,Um general romano planeja a melhor vingança co...


In [16]:
df['sinopse'] = df['sinopse'].str.lower()
df

,title_pt,title_en,year,rating,genre,sinopse
0,O Poderoso Chefão II,4. The Godfather Part II,1974,9.0,Policial,"em 1950, michael corleone, agora à frente da f..."
1,12 Homens e uma Sentença,5. 12 Angry Men,1957,9.0,Policial,o julgamento de um assassinato em nova iorque ...
2,A Lista de Schindler,6. Schindler's List,1993,9.0,Biografia,na polônia ocupada pelos alemães durante a seg...
3,O Senhor dos Anéis: O Retorno do Rei,7. The Lord of the Rings: The Return of the King,2003,9.0,Ação,gandalf e aragorn lideram o mundo dos homens c...
4,O Senhor dos Anéis: A Sociedade do Anel,9. The Lord of the Rings: The Fellowship of th...,2001,8.8,Ação,um manso hobbit do condado e oito companheiros...
...,...,...,...,...,...,...
430,Tudo em Todo o Lugar ao Mesmo Tempo,Everything Everywhere All at Once,2022,7.8,Ação,uma imigrante chinesa de meia idade se envolve...
431,Homem-aranha: Lotus,Spider-Man: Lotus,2023,3.4,Ação,"após a morte de sua namorada, peter parker se ..."
433,O Rei Macaco,The Monkey King,2023,5.8,Animação,segue um macaco cheio de carisma e seu bastão ...
434,Gladiador,Gladiator,2000,8.5,Ação,um general romano planeja a melhor vingança co...


In [17]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'é',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'éramos',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'está',
 'estamos',
 'estão',
 'estar',
 'estas',
 'estava',
 'estavam',
 'estávamos',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estivéramos',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéssemos',
 'estou',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'fôramos',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fôssemos',
 'fui',
 'há',
 'haja',
 'hajam',
 'hajamos',
 'hão',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houverá',
 'houveram',
 'houvéramos',
 'houverão',
 'houverei',
 'houverem',
 'hou

In [18]:
df['sinopse'] = df['sinopse'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
df.head()

,title_pt,title_en,year,rating,genre,sinopse
0,O Poderoso Chefão II,4. The Godfather Part II,1974,9.0,Policial,"1950, michael corleone, agora frente família, ..."
1,12 Homens e uma Sentença,5. 12 Angry Men,1957,9.0,Policial,julgamento assassinato nova iorque frustrado ú...
2,A Lista de Schindler,6. Schindler's List,1993,9.0,Biografia,polônia ocupada alemães durante segunda guerra...
3,O Senhor dos Anéis: O Retorno do Rei,7. The Lord of the Rings: The Return of the King,2003,9.0,Ação,gandalf aragorn lideram mundo homens contra ex...
4,O Senhor dos Anéis: A Sociedade do Anel,9. The Lord of the Rings: The Fellowship of th...,2001,8.8,Ação,manso hobbit condado oito companheiros partem ...


In [39]:

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['sinopse'])
tfidf_matrix.toarray()


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.26303395,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.22652041, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [42]:
s = setup(tfidf_matrix.toarray(), session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(422, 3409)"
2,Transformed data shape,"(422, 3409)"
3,Numeric features,3409
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [43]:
knn = create_model('knn')

In [34]:


# Fit NearestNeighbors using the TF-IDF matrix
nbrs = NearestNeighbors(n_neighbors=11).fit(tfidf_matrix)

# Get the TF-IDF vector for a specific movie
movie_index = df[df['title_en'] == 'The Godfather Part II'].index[0]
movie_vector = tfidf_matrix[movie_index]

# Retrieve the indices of the 10 most similar movies
distances, indices = nbrs.kneighbors(movie_vector)

# Print the titles of the similar movies
for index in indices[0][1:]:
    print(df.iloc[index]['title_en'])
